In [46]:
# Creating heatmaps of frontier sector firms in London
# Author: Will Shepherd
# Date: November 2025

import pandas as pd
import geopandas as gpd
import numpy as np
import altair as alt
import os
import eco_style 
alt.themes.enable("light")

ThemeRegistry.enable('light')

In [36]:
conda install vl-convert-python

2 channel Terms of Service accepted
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


OLD DATA - DEFINITIONS FOR FRONTIER SECTOR HAVE NOW BEEN UPDATED

In [47]:
# Read in data for London LADs and MSOAs
lad_df = pd.read_excel('London_FrontierSectorFirms_BSD_2023.xlsx', sheet_name='LAD11')
msoa_df = pd.read_excel('London_FrontierSectorFirms_BSD_2023.xlsx', sheet_name='MSOA11')

In [48]:
# Calculate firm and employment shares for LAD

lad_df['frontier_firm_share'] = lad_df['n_frontier_firms'] / lad_df['n_firms']
lad_df['climate_tech_firm_share'] = lad_df['n_climate_tech_firms'] / lad_df['n_firms']
lad_df['advanced_manu_firm_share'] = lad_df['n_advanced_manu_firms'] / lad_df['n_firms']
lad_df['digital_firm_share'] = lad_df['n_digital_firms'] / lad_df['n_firms']
lad_df['finance_firm_share'] = lad_df['n_finance_firms'] / lad_df['n_firms']
lad_df['lifesci_firm_share'] = lad_df['n_lifesci_firms'] / lad_df['n_firms']

lad_df['frontier_emp_share'] = lad_df['emp_frontier'] / lad_df['total_emp']
lad_df['climate_tech_emp_share'] = lad_df['emp_climate'] / lad_df['total_emp']
lad_df['advanced_manu_emp_share'] = lad_df['emp_adv_manu'] / lad_df['total_emp']
lad_df['digital_emp_share'] = lad_df['emp_digital'] / lad_df['total_emp']
lad_df['finance_emp_share'] = lad_df['emp_finance'] / lad_df['total_emp']
lad_df['lifesci_emp_share'] = lad_df['emp_lifesci'] / lad_df['total_emp']

# Calculate firm and employment shares for MSOA
msoa_df['n_frontier_firms'] = pd.to_numeric(msoa_df['n_frontier_firms'], errors='coerce')
msoa_df['emp_frontier'] = pd.to_numeric(msoa_df['emp_frontier'], errors='coerce')

msoa_df['frontier_firm_share'] = msoa_df['n_frontier_firms'] / msoa_df['n_firms']
msoa_df['frontier_emp_share'] = msoa_df['emp_frontier'] / msoa_df['total_emp']



In [49]:
# First read in LAD and MSOA shapefiles with geopandas
lad_shapefile_path = "Local_Authority_Districts_December_2011_FEB_EW_2022_-327262821801797584/Local_Authority_Districts_December_2011_FEB_EW.shp"
msoa_shapefile_path = "MSOA_Dec_2011_Boundaries_Generalised_Clipped_BGC_EW_V3_2022_-8564488481746373263/MSOA_2011_EW_BGC_V3.shp"

lad_shp = gpd.read_file(lad_shapefile_path)
msoa_shp = gpd.read_file(msoa_shapefile_path)

In [50]:
# LAD11 - Inner join the LAD shapefile with the LONDON data so we just have rows for London local authorities
london_lad_gdf = lad_shp.merge(
    lad_df,
    on='lad11nm',
    how='inner'
)
if london_lad_gdf.crs != "EPSG:4326":
    print("Converting CRS to EPSG:4326...")
    london_lad_gdf = london_lad_gdf.to_crs(epsg=4326)
    print("Conversion complete.")

alt.data_transformers.enable('json')

Converting CRS to EPSG:4326...
Conversion complete.


DataTransformerRegistry.enable('json')

In [51]:
# MSOA11 - Inner join the MSOA shapefile with the LONDON data so we just have rows for London MSOA
msoa_df = msoa_df.rename(columns={'msoa11cd':'MSOA11CD'})

london_msoa_gdf = msoa_shp.merge(
    msoa_df,
    on='MSOA11CD',
    how='inner'
)
if london_msoa_gdf.crs != "EPSG:4326":
    print("Converting CRS to EPSG:4326...")
    london_msoa_gdf = london_msoa_gdf.to_crs(epsg=4326)
    print("Conversion complete.")

alt.data_transformers.enable('json')

Converting CRS to EPSG:4326...
Conversion complete.


DataTransformerRegistry.enable('json')

In [52]:
# PLOT FRONTIER FIRM SHARE AND EMPLOYMENT SHARE SIDE-BY-SIDE FOR LONDON MSOAs

# Define the variables to plot
msoa_variables = {'frontier_firm_share':'Frontier firm share',
                  'frontier_emp_share':'Frontier employment share'}

# Create an empty list to hold the charts
chart_list = []

# For each variable, create a separate chart
for column_name, friendly_title in msoa_variables.items():

    chart = alt.Chart(london_msoa_gdf).mark_geoshape(
    stroke='black',    
    strokeWidth=0.5    
    ).encode(
    color=alt.Color(
        f'{column_name}:Q', 
        title=None,
        legend=alt.Legend(
            orient="none",
            direction='horizontal',
            legendY=620,
            legendX=220,
            format='%'
    )),
    tooltip=[
        'MSOA11NM:N',  
        alt.Tooltip(
                f'{column_name}',
                type='quantitative',
                format='.0%'
            )
    ]
    ).properties(
    title={
        "text": friendly_title + " in London MSOAs",
        "anchor": "middle",  
        "dy": 100,
        "fontSize": 18
    },
    width=600,
    height=700
    )

    # Add the newly created chart to our list
    chart_list.append(chart)

# Combine both charts
if len(chart_list) == 2:
    final_grid = alt.vconcat(
        alt.hconcat(chart_list[0], chart_list[1]).resolve_scale(
        color='independent'
    ).resolve_legend(
        color='independent'
    )
    )
else:
    print("Warning: Expected 2 charts, but found {len(chart_list)}. Displaying vertically.")
    # Fallback: just stack them vertically
    final_grid = alt.vconcat(*chart_list)

final_grid

# Save the final grid as PNG and JSON
final_grid.save('Maps/frontier_sector_firms_LDN_MSOA.png', scale_factor=2)
final_grid.save('Maps/frontier_sector_firms_LDN_MSOA.json')


In [53]:
# PLOT SHARE OF FIRMS IN EACH FRONTIER SECTOR FOR LONDON LADS
# 6 PLOTS IN A 2x3 GRID

# Select variables to plot and include friendly titles for plotting
lad_variables = {'frontier_firm_share':'Share of frontier sector firms',
                 'climate_tech_firm_share':'Share of climate tech firms',
                 'advanced_manu_firm_share':'Share of advanced manufacturing firms',
                 'digital_firm_share':'Share of digital firms',
                 'finance_firm_share':'Share of financial firms',
                 'lifesci_firm_share':'Share of life science firms'}

# Create an empty list to hold the individual charts
chart_list = []

for column_name, friendly_title in lad_variables.items():
    
    # Create the chart for this variable (looping through each in list)
    chart = alt.Chart(london_lad_gdf).mark_geoshape(
        stroke='black',
        strokeWidth=0.5
    ).encode(
        color=alt.Color(f'{column_name}:Q', title=None,
                        legend=alt.Legend(
                orient="none",
                legendY=250,
                legendX=-100,
                format='%'
        )),
        
        tooltip=[
            'lad11nm:N', 
            f'{column_name}:Q'
        ]
    ).properties(
        title=friendly_title, 
        width=300,  
        height=350 
    ).project(
        type='transverseMercator'
    )
    
    # Add the newly created chart to our list
    chart_list.append(chart)

# Combine all 6 charts into a 2x3 grid
if len(chart_list) == 6:
    final_grid = alt.vconcat(
        alt.hconcat(chart_list[0], chart_list[1], chart_list[2]),
        alt.hconcat(chart_list[3], chart_list[4], chart_list[5]).resolve_scale(
        color='independent'
    ).resolve_legend(
        color='independent'
    )
    )
else:
    print("Warning: Expected 6 charts, but found {len(chart_list)}. Displaying vertically.")
    # Fallback: just stack them vertically
    final_grid = alt.vconcat(*chart_list)

# Save the final grid of charts
final_grid.save('Maps/frontier_sector_firmshare_LDN_LAD.png', scale_factor=2)
final_grid.save('Maps/frontier_sector_firmshare_LDN_LAD.json')


In [45]:
# PLOT SHARE OF EMPLOYMENT IN EACH FRONTIER SECTOR FOR LONDON LADS
# 6 PLOTS IN A 2x3 GRID

lad_employment_variables = {'frontier_emp_share':'Share of employment in frontier sectors',
                 'climate_tech_emp_share':'Share of employment in climate tech',
                 'advanced_manu_emp_share':'Share of employment in advanced manufacturing',
                 'digital_emp_share':'Share of employment in digital',
                 'finance_emp_share':'Share of employment in finance',
                 'lifesci_emp_share':'Share of employment in life sciences'}

chart_list = []

for column_name, friendly_title in lad_employment_variables.items():
    
    # Create the chart for this variable
    chart = alt.Chart(london_lad_gdf).mark_geoshape(
        stroke='black',
        strokeWidth=0.5
    ).encode(
        color=alt.Color(f'{column_name}:Q', title=None,
                        legend=alt.Legend(
                orient="none",
                legendY=250,
                legendX=-100,
                format='%'
        )),
        
        tooltip=[
            'lad11nm:N', 
            f'{column_name}:Q'
        ]
    ).properties(
        title=friendly_title, 
        width=300,  
        height=350 
    ).project(
        type='transverseMercator'
    )
    
    # Add the newly created chart to our list
    chart_list.append(chart)

# Combine all 6 charts into a 2x3 grid
if len(chart_list) == 6:
    final_grid = alt.vconcat(
        alt.hconcat(chart_list[0], chart_list[1], chart_list[2]),
        alt.hconcat(chart_list[3], chart_list[4], chart_list[5])
    )
else:
    print("Warning: Expected 6 charts, but found {len(chart_list)}. Displaying vertically.")
    # Fallback: just stack them vertically
    final_grid = alt.vconcat(*chart_list)

# Save the final grid
final_grid.save('Maps/frontier_sector_emp_share_LDN_LAD.png', scale_factor=2)
final_grid.save('Maps/frontier_sector_emp_share_LDN_LAD.json')

FROM THIS POINT ON THE UPDATED FRONTIER SECTOR DEFINITIONS ARE USED - THIS WAS ONLY EXPORTED AT THE MSOA LEVEL

In [ ]:
# Import updated MSOA level frontier firm data
msoa_df